### Training code for dataset Bla Bla

Description of the dataset, experiment etc

### Important ! 

This step can be skipped! Only run this notebook if you want to train the microsplit model from scratch or finetune. Pretrained model checkpoint is available 

#### General imports

In [ ]:
import pooch
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader
from careamics.lightning import VAEModule
from careamics.lvae_training.dataset import DataSplitType

from microsplit_reproducibility.configs.factory import (
    create_algorithm_config,
    get_likelihood_config,
    get_loss_config,
    get_model_config,
    get_optimizer_config,
    get_training_config,
    get_lr_scheduler_config,
)
from microsplit_reproducibility.utils.callbacks import get_callbacks
from microsplit_reproducibility.utils.io import load_checkpoint
from microsplit_reproducibility.datasets import create_train_val_datasets
from microsplit_reproducibility.utils.utils import plot_training_metrics

#### Experiments specific imports

In [ ]:
from microsplit_reproducibility.configs.parameters.HT_LIF24 import get_microsplit_parameters
from microsplit_reproducibility.configs.data.HT_LIF24 import get_data_configs
from microsplit_reproducibility.datasets.HT_LIF24 import get_train_val_data

### Get data and experiment parameters

Example training code 5 epochs, switch between full training, short training,  fine-tuning 

In [ ]:
train_data_config, val_data_config, test_data_configs = get_data_configs(dset_type="20ms")
experiment_params = get_microsplit_parameters(dset_type="20ms")

In [ ]:
for k in train_data_config:
    print(k)    

In [ ]:
experiment_params

### Download the data

In [ ]:
DATA = pooch.create(
    path="./data",
    base_url="https://download.fht.org/jug/ht_lif24",
    registry={"ht_lif24.zip": None},
)

NOISE_MODELS = pooch.create(
    path="./noise_models",
    base_url="https://download.fht.org/jug/ht_lif24",
    registry={"nm_ht_lif24_ch1_20ms.npz": None,
              "nm_ht_lif24_ch2_20ms.npz": None,
              "nm_ht_lif24_ch3_20ms.npz": None},
)

MODEL_CHECKPOINTS = pooch.create(
    path="./checkpoints",
    base_url="https://download.fht.org/jug/ht_lif24",
    registry={"best.ckpt": None,
              "last.ckpt": None},
)

In [ ]:
for i, f in enumerate(NOISE_MODELS.registry):
    NOISE_MODELS.fetch(f"nm_ht_lif24_ch{i+1}_20ms.npz")

DATA.fetch("ht_lif24.zip", processor=pooch.Unzip())

for f in MODEL_CHECKPOINTS.registry:
    MODEL_CHECKPOINTS.fetch(f"{f}")


### Create dataset

In [ ]:
train_dset, val_dset, _, data_stats = create_train_val_datasets(
    datapath=DATA.path / "ht_lif24.zip.unzip/ht_lif24",
    train_config=train_data_config,
    val_config=val_data_config,
    test_config=val_data_config,
    load_data_func=get_train_val_data,
)

### Create dataloaders

In [ ]:
train_dloader = DataLoader(
    train_dset,
    batch_size=experiment_params["batch_size"],
    num_workers=experiment_params["num_workers"],
    shuffle=True,
)
val_dloader = DataLoader(
    val_dset,
    batch_size=experiment_params["batch_size"],
    num_workers=experiment_params["num_workers"],
    shuffle=False,
)

### Visualize input data

In [ ]:
# TODO grid of input patches 

### Get experiment configs

In [ ]:
experiment_params["data_stats"] = data_stats # TODO rethink

loss_config = get_loss_config(**experiment_params)
model_config = get_model_config(**experiment_params)
gaussian_lik_config, noise_model_config, nm_lik_config = get_likelihood_config(
    **experiment_params
)
training_config = get_training_config(**experiment_params)
lr_scheduler_config = get_lr_scheduler_config(**experiment_params)
optimizer_config = get_optimizer_config(**experiment_params)

experiment_config = create_algorithm_config(
    algorithm=experiment_params["algorithm"],
    loss_config=loss_config,
    model_config=model_config,
    gaussian_lik_config=gaussian_lik_config,
    nm_config=noise_model_config,
    nm_lik_config=nm_lik_config,
    lr_scheduler_config=lr_scheduler_config,
    optimizer_config=optimizer_config,
)

In [ ]:
experiment_config


### Initialize the model

In [ ]:
model = VAEModule(algorithm_config=experiment_config)

### Load checkpoint (optional)

It's possible to load a checkpoint to continue training

In [ ]:
ckpt = load_checkpoint("checkpoints", best=False)
model = VAEModule.load_from_checkpoint(ckpt, algorithm_config=experiment_config) # TODO fix seek error

### Train the model

Only 5 epochs for the sake of the example

In [ ]:
trainer = Trainer(
    max_epochs=training_config.num_epochs,
    accelerator="gpu",
    enable_progress_bar=True,
    callbacks=get_callbacks("."),
    precision=training_config.precision,
    gradient_clip_val=training_config.gradient_clip_val,
    gradient_clip_algorithm=training_config.gradient_clip_algorithm,
)
trainer.fit(
    model=model,
        train_dataloaders=train_dloader,
        val_dataloaders=val_dloader,
    )

### Training logs

In [ ]:
plot_training_metrics(f"csv_logs/{experiment_params['experiment_name']}/version_0/")

In [ ]:
# TODO grid of predictions of last epoch, input/ k channels